In [ ]:
# Dataset
import os
import shutil
import numpy as np
import nibabel as nib
import tensorflow as tf
import cv2

input_dir = './Dataset'
output_dir = './Dataset/Processed'
total_size={'Training':369,'Validation':125}
modalities=['flair', 't1', 't1ce', 't2']

def load_nifti_file(filepath):
    #Load a NIfTI file and return its data as a numpy array
    scan = nib.load(filepath)
    return scan.get_fdata()

def load_data(phase,chunk_index,chunk_size,input_dir, output_dir):
        start_id = chunk_index * chunk_size + 1
        end_id = min((chunk_index + 1) * chunk_size, total_size[phase])
        for i in range(start_id,end_id+1):
            id = f"{i:03d}"
            npy_folder_path = os.path.join(output_dir, phase, f'chunk_{chunk_index}',id)
            os.makedirs(npy_folder_path, exist_ok=True)
            for modality in modalities:
                #Modality data
                channel_name = f'BraTS20_{phase}_{id}_{modality}'
                channel_path = os.path.join(input_dir, phase, modality)
                channel = load_nifti_file(os.path.join(channel_path, channel_name+ '.nii'))
                # Save numpy channel file
                npy_channel_file_path=os.path.join(npy_folder_path,channel_name+ '.npy')
                np.save(npy_channel_file_path, channel)

            #Mask data
            mask_path= os.path.join(input_dir, phase, 'mask')
            mask_name= f'BraTS20_{phase}_{id}_{'seg'}'
            mask = load_nifti_file(os.path.join(mask_path, mask_name + '.nii'))
            # Save numpy mask file
            npy_mask_file_path = os.path.join(npy_folder_path, mask_name + '.npy')
            np.save(npy_mask_file_path, mask)
            
X = np.empty((0, 128, 128, 4))
y = np.empty((0, 128, 128, 4), dtype=int)
for index in range(9):
    chunk_size=41
    phase='Training'
    load_data(phase=phase,chunk_index=index,chunk_size=chunk_size,input_dir=input_dir, output_dir=output_dir)
    chunk_index=index
    start_id = chunk_index * chunk_size + 1
    end_id = min((chunk_index + 1) * chunk_size, total_size[phase])
    x = np.empty([chunk_size * 155, 128, 128, 4])
    y_tr = np.empty([chunk_size*155,128, 128,4], dtype=int)
    # training
    for i in range(start_id,end_id+1):
        id = f"{i:03d}"
        numpy_folder_path = os.path.join(output_dir, phase, f'chunk_{chunk_index}',id)
        for h in range(155):
            # y
            npy_mask=np.load(os.path.join(numpy_folder_path, f'BraTS20_{phase}_{id}_{'seg'}' + '.npy'))
            mask = tf.one_hot(npy_mask[:, :, h], 4)  # One-hot encoding
            Y = tf.image.resize(mask, (128, 128))  # Resize
            y_tr[(i - start_id) * 155 + h] = Y.numpy()
            # X_train
            for k,modality in enumerate(modalities):
                nmpy_channel= np.load(os.path.join(numpy_folder_path,f'BraTS20_{phase}_{id}_{modality}'+ '.npy'))
                x[(i - start_id) * 155 + h , :, :, k] = cv2.resize(nmpy_channel[:, :, h], (128, 128))

    y_tr[y_tr==4] = 3
    
    # Concatenate the current chunk with the existing data
    X = np.concatenate((X, x), axis=0)
    y = np.concatenate((y, y_tr), axis=0)
    shutil.rmtree(os.path.join(output_dir, phase, f'chunk_{chunk_index}'))


print(f'Final X shape={X.shape}, y shape={y.shape}')

In [ ]:
import os
import shutil
import numpy as np
import nibabel as nib
import tensorflow as tf
import cv2

class NiftiProcessor:
    def __init__(self, input_dir, output_dir, total_size, modalities):
        self.input_dir = input_dir
        self.output_dir = output_dir
        self.total_size = total_size
        self.modalities = modalities
        self.X = np.empty((0, 128, 128, 4))
        self.y = np.empty((0, 128, 128, 4), dtype=int)

    def load_nifti_file(self, filepath):
        # Load a NIfTI file and return its data as a numpy array
        scan = nib.load(filepath)
        return scan.get_fdata()

    def load_data(self, phase, chunk_index, chunk_size):
        start_id = chunk_index * chunk_size + 1
        end_id = min((chunk_index + 1) * chunk_size, self.total_size[phase])
        for i in range(start_id, end_id + 1):
            id = f"{i:03d}"
            npy_folder_path = os.path.join(self.output_dir, phase, f'chunk_{chunk_index}', id)
            os.makedirs(npy_folder_path, exist_ok=True)
            for modality in self.modalities:
                # Modality data
                channel_name = f'BraTS20_{phase}_{id}_{modality}'
                channel_path = os.path.join(self.input_dir, phase, modality)
                channel = self.load_nifti_file(os.path.join(channel_path, channel_name + '.nii'))
                # Save numpy channel file
                npy_channel_file_path = os.path.join(npy_folder_path, channel_name + '.npy')
                np.save(npy_channel_file_path, channel)

            # Mask data
            mask_path = os.path.join(self.input_dir, phase, 'mask')
            mask_name = f'BraTS20_{phase}_{id}_seg'
            mask = self.load_nifti_file(os.path.join(mask_path, mask_name + '.nii'))
            # Save numpy mask file
            npy_mask_file_path = os.path.join(npy_folder_path, mask_name + '.npy')
            np.save(npy_mask_file_path, mask)

    def process_data(self, phase, chunk_index, chunk_size):
        start_id = chunk_index * chunk_size + 1
        end_id = min((chunk_index + 1) * chunk_size, self.total_size[phase])
        x = np.empty([chunk_size * 155, 128, 128, 4])
        y_tr = np.empty([chunk_size * 155, 128, 128, 4], dtype=int)
        for i in range(start_id, end_id + 1):
            id = f"{i:03d}"
            numpy_folder_path = os.path.join(self.output_dir, phase, f'chunk_{chunk_index}', id)
            for h in range(155):
                # y
                npy_mask = np.load(os.path.join(numpy_folder_path, f'BraTS20_{phase}_{id}_seg.npy'))
                mask = tf.one_hot(npy_mask[:, :, h], 4)  # One-hot encoding
                Y = tf.image.resize(mask, (128, 128))  # Resize
                y_tr[(i - start_id) * 155 + h] = Y.numpy()
                # X_train
                for k, modality in enumerate(self.modalities):
                    nmpy_channel = np.load(os.path.join(numpy_folder_path, f'BraTS20_{phase}_{id}_{modality}.npy'))
                    x[(i - start_id) * 155 + h, :, :, k] = cv2.resize(nmpy_channel[:, :, h], (128, 128))
                print((i - start_id) * 155 + h)
        y_tr[y_tr == 4] = 3
        self.X = np.concatenate((self.X, x), axis=0)
        self.y = np.concatenate((self.y, y_tr), axis=0)
        shutil.rmtree(os.path.join(self.output_dir, phase, f'chunk_{chunk_index}'))

    def run(self, phase, num_chunks, chunk_size):
        for index in range(num_chunks):
            print(index)
            self.load_data(phase=phase, chunk_index=index, chunk_size=chunk_size)
            self.process_data(phase=phase, chunk_index=index, chunk_size=chunk_size)
        print(f'Final X shape={self.X.shape}, y shape={self.y.shape}')


input_dir = './Dataset'
output_dir = './Dataset/Processed'
total_size = {'Training': 369, 'Validation': 125}
modalities = ['flair', 't1', 't1ce', 't2']
num_chunks = 3
chunk_size = 41

processor = NiftiProcessor(input_dir, output_dir, total_size, modalities)
processor.run(phase='Training', num_chunks=num_chunks, chunk_size=chunk_size)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,shuffle=True,test_size=0.2)

In [ ]:
# Unsupervised validation set
X_validation= np.empty([0,128, 128,4])
    for chunk_index in range(2):
        chunk_size=10
        phase='Validation'
        start_id = chunk_index * chunk_size + 1
        end_id = min((chunk_index + 1) * chunk_size, total_size[phase])
        X_val = np.empty([chunk_size*155,128, 128,4])
        # validation
        for i in range(start_id,end_id+1):
            id = f"{i:03d}"
            numpy_folder_path = os.path.join(output_dir, phase, f'chunk_{chunk_index}',id)
            for h in range(155):
                # X_val
                for k,modality in enumerate(modalities):
                    nmpy_channel= np.load(os.path.join(numpy_folder_path,f'BraTS20_{phase}_{id}_{modality}'+ '.npy'))
                    X_val[(i - start_id) * 155 + h , :, :, k] = cv2.resize(nmpy_channel[:, :, h], (128, 128))        
        # Concatenate the current chunk with the existing data
        X_validation = np.concatenate((X, X_train), axis=0)

In [ ]:

# def train_datagen(phase,chunk_index,chunk_size,X_train,y_train):
#         start_id = chunk_index * chunk_size + 1
#         end_id = min((chunk_index + 1) * chunk_size, total_size[phase])
#         # training
#         for i in range(start_id,end_id):
#             id = f"{i:03d}"
#             numpy_folder_path = os.path.join(output_dir, phase, f'chunk_{chunk_index}',id)
#             for h in range(155):
#                 # y
#                 npy_mask=np.load(os.path.join(numpy_folder_path, f'BraTS20_{phase}_{id}_{'seg'}' + '.npy'))
#                 y_train[h+155*chunk_index] = npy_mask[:, :, h]
#                 # X_train
#                 for k,modality in enumerate(modalities):
#                     nmpy_channel= np.load(os.path.join(numpy_folder_path,f'BraTS20_{phase}_{id}_{modality}'+ '.npy'))
#                     X_train[h+155*chunk_index , :, :, k] = cv2.resize(nmpy_channel[:, :, h], (128, 128))
# 
#         y_train[y_train==4] = 3
#         # mask = tf.one_hot(y_train, 4)
#         # Y_train = tf.image.resize(mask, (128, 128))
#         return X_train/np.max(X_train), Y_train

In [ ]:
# Model Artichecture


In [ ]:
SEGMENT_CLASSES = {
    0 : 'NOT tumor',
    1 : 'NECROTIC/CORE', # or NON-ENHANCING tumor CORE
    2 : 'EDEMA',
    3 : 'ENHANCING' # original 4 -> converted into 3 later
}
input_layer = Input((128, 128, 4))
model = build_unet(input_layer, 'he_normal', 0.2)
model.compile(
    loss="categorical_crossentropy", 
    optimizer=keras.optimizers.Adam(learning_rate=0.001), 
    metrics=[
        'accuracy',
        tf.keras.metrics.MeanIoU(num_classes=4),
        dice_coef,
        precision,
        sensitivity,
        specificity,
        dice_coef_necrotic,
        dice_coef_edema,
        dice_coef_enhancing
    ]
)
plot_model(model, 
           show_shapes = True,
           show_dtype=False,
           show_layer_names = True, 
           rankdir = 'TB', 
           expand_nested = False, 
           dpi = 70)


In [ ]:
# Train 

In [ ]:
# Evaluate

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, Concatenate, Activation
from tensorflow.keras.models import Model

def conv_block(inputs, num_filters):
    """Convolutional block consisting of two Conv3D layers followed by an activation function."""
    x = Conv3D(num_filters, 3, padding='same')(inputs)
    x = Activation('relu')(x)
    x = Conv3D(num_filters, 3, padding='same')(x)
    x = Activation('relu')(x)
    return x

def encoder_block(inputs, num_filters):
    """Encoder block consisting of a conv block followed by a max pooling layer."""
    x = conv_block(inputs, num_filters)
    p = MaxPooling3D(pool_size=(2, 2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    """Decoder block consisting of an upsampling layer followed by a conv block."""
    x = UpSampling3D(size=(2, 2, 2))(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x
def unet_model(input_shape):
    """Builds the 3D U-Net model."""
    inputs = Input(input_shape)

    # Encoder
    s1, p1 = encoder_block(inputs, 32)
    s2, p2 = encoder_block(p1, 64)
    s3, p3 = encoder_block(p2, 128)
    s4, p4 = encoder_block(p3, 256)

    # Bridge
    b1 = conv_block(p4, 512)

    # Decoder
    d1 = decoder_block(b1, s4, 256)
    d2 = decoder_block(d1, s3, 128)
    d3 = decoder_block(d2, s2, 64)
    d4 = decoder_block(d3, s1, 32)

    outputs = Conv3D(1, 1, padding='same', activation='sigmoid')(d4)

    model = Model(inputs, outputs, name='3d_unet')
    return model
model = unet_model(input_shape=(*image_size, 1))
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

checkpoint = ModelCheckpoint('unet_brats2020.h5', save_best_only=True, monitor='val_loss', mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min')

history = model.fit(train_gen, validation_data=val_gen, epochs=epochs, callbacks=[checkpoint, early_stopping])
model.save_weights('model_weights.h5')

In [ ]:
# Iterate over each chunk
for start in range(1, total_files + 1, chunk_size):
    Datagen(phase='Training', chunk_index=start, chunk_size=chunk_size, input_dir=input_dir, output_dir=output_dir)
    end = min(start_id + chunk_size - 1, total_files)